# Stripping BAM files of sequence/quality information

BAM files contain the alignment information for reads mapping
to a genome. This information includes the position a read
matches, various flags and quality metrics of the match, but also
the sequence of a read and the base quality scores of a read.

The read sequence and base quality scores are essential for deduping and
for applications such as variant calling. But once your are think 
that your BAM-files contains only reads that have been mapped with
confidence, the sequence and base quality scores are not required
for tag counting applications such as RNA-seq or ChIP-seq and can
be discarded.

The mapped  positions could be converted into a bed file, but for workflows
requiring bam-files, the *bam2bam* utility permits removing
read sequences and quality scores directly.

The following command will remove the sequence and quality information from the file rnaseq_hg19_chr19.bam

In [1]:
!cgat bam2bam --strip-method=all --method=strip-sequence --log=strip.log < rnaseq_hg19_chr19.bam > stripped.bam

In [2]:
!ls -hlL rnaseq_hg19_chr19.bam stripped.bam

-rw-rw-r-- 1 andreas usersfgu 788M Apr  2  2014 rnaseq_hg19_chr19.bam
-rw-rw-r-- 1 andreas usersfgu 215M Apr 24 14:20 stripped.bam


The size of the BAM files has dropped from 788M to 215M. The bam-file can be used for further processing.

In [1]:
!samtools flagstat stripped.bam

14998038 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
14998038 + 0 mapped (100.00% : N/A)
14998038 + 0 paired in sequencing
7388885 + 0 read1
7609153 + 0 read2
13659120 + 0 properly paired (91.07% : N/A)
14745975 + 0 with itself and mate mapped
252063 + 0 singletons (1.68% : N/A)
1086855 + 0 with mate mapped to a different chr
1086855 + 0 with mate mapped to a different chr (mapQ>=5)


Nothing has changed compared to our original bam file

In [2]:
!samtools flagstat rnaseq_hg19_chr19.bam

14998038 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
14998038 + 0 mapped (100.00% : N/A)
14998038 + 0 paired in sequencing
7388885 + 0 read1
7609153 + 0 read2
13659120 + 0 properly paired (91.07% : N/A)
14745975 + 0 with itself and mate mapped
252063 + 0 singletons (1.68% : N/A)
1086855 + 0 with mate mapped to a different chr
1086855 + 0 with mate mapped to a different chr (mapQ>=5)


Some tools might fail if there is no sequence information in the bam file.

In [2]:
!cat stripped.bam | /ifs/apps/bio/picard-tools-2.8.3/CollectMultipleMetrics INPUT=/dev/stdin OUTPUT=metrics.txt

[Mon Apr 24 14:49:39 BST 2017] picard.analysis.CollectMultipleMetrics INPUT=/dev/stdin OUTPUT=metrics.txt    ASSUME_SORTED=true STOP_AFTER=0 METRIC_ACCUMULATION_LEVEL=[ALL_READS] PROGRAM=[CollectAlignmentSummaryMetrics, CollectBaseDistributionByCycle, CollectInsertSizeMetrics, MeanQualityByCycle, QualityScoreDistribution] INCLUDE_UNPAIRED=false VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false GA4GH_CLIENT_SECRETS=client_secrets.json
[Mon Apr 24 14:49:39 BST 2017] Executing as andreas@cgat055.anat.ox.ac.uk on Linux 2.6.32-642.15.1.el6.x86_64 amd64; Java HotSpot(TM) 64-Bit Server VM 1.8.0_131-b11; Picard version: 2.8.3-SNAPSHOT
WARNING	2017-04-24 14:49:39	SinglePassSamProgram	File reports sort order 'unsorted', assuming it's coordinate sorted anyway.
[Mon Apr 24 14:49:39 BST 2017] picard.analysis.CollectMultipleMetrics done. Elapsed time: 0.00 minutes.
Runtime.totalMemory()=508887040
To get help

In this case, the sequence information can be added back either with dummy sequence:

In [4]:
!cat rnaseq_hg19_chr19.bam | cgat bam2bam -v 0 --method=set-sequence - | /ifs/apps/bio/picard-tools-2.8.3/CollectMultipleMetrics INPUT=/dev/stdin OUTPUT=metrics.txt VALIDATION_STRINGENCY=SILENT

[Mon Apr 24 14:50:37 BST 2017] picard.analysis.CollectMultipleMetrics INPUT=/dev/stdin OUTPUT=metrics.txt VALIDATION_STRINGENCY=SILENT    ASSUME_SORTED=true STOP_AFTER=0 METRIC_ACCUMULATION_LEVEL=[ALL_READS] PROGRAM=[CollectAlignmentSummaryMetrics, CollectBaseDistributionByCycle, CollectInsertSizeMetrics, MeanQualityByCycle, QualityScoreDistribution] INCLUDE_UNPAIRED=false VERBOSITY=INFO QUIET=false COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false GA4GH_CLIENT_SECRETS=client_secrets.json
[Mon Apr 24 14:50:37 BST 2017] Executing as andreas@cgat055.anat.ox.ac.uk on Linux 2.6.32-642.15.1.el6.x86_64 amd64; Java HotSpot(TM) 64-Bit Server VM 1.8.0_131-b11; Picard version: 2.8.3-SNAPSHOT
WARNING	2017-04-24 14:50:38	SinglePassSamProgram	File reports sort order 'unsorted', assuming it's coordinate sorted anyway.
INFO	2017-04-24 14:50:46	SinglePassSamProgram	Processed     1,000,000 records.  Elapsed time: 00:00:08s.  Time for last 1,000,000:    8s.  Last rea

And voila, picard works. Of course, some metrics will not be meaningful without the read sequences and base quality scores. The original sequence can be added back using the *unstrip* option. Note that this is operation might take some
time and require a substantial amount of memory. It is thus best left for debugging purposes only. If sequence is required in a BAM file, it is best to leave it in.

In [5]:
!cgat bam2bam --method=unstrip -1 rnaseq_hg19_chr19.fastq.1.gz -2 rnaseq_hg19_chr19.fastq.2.gz --log=unstrip.log < stripped.bam > unstripped.bam

To check if we got our original file back, we can compare the sequence contents. Note that the files might not be binary equal.
To save time, we are checking only if the first 1000 lines are equal (ignore the errors resulting from closing the samtools pipe after 1000 lines).

In [11]:
!samtools view unstripped.bam | head -n 1000 > a
!samtools view rnaseq_hg19_chr19.bam | head -n 1000 > b
!wc -l a b
!echo "diff is:"
!diff a b
!echo "cleaning up"
!rm -f a b

samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1
samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1
  1000 a
  1000 b
  2000 total
diff is:
cleaning up
